In [1]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
import pickle
import time
import tensorflow as tf
import shutil
import numpy as np
from math import ceil
from src import constants as const
import utils.timer
import helpers

dataset_train_path = os.path.join(const.dir.data(), "df_adult_edited_train.csv")
dataset_test_path = os.path.join(const.dir.data(), "df_adult_edited_test.csv")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
columns_num, = np.where(data_train.dtypes != "object")

timer_path = os.path.join(const.dir.timers(), "timer_tgan.pkl")
retrain = True

if not os.path.exists(timer_path):
    timer = utils.timer.Timer()
    timer.save(timer_path)
    del timer
    
from tgan.model import TGANModel

dir_data_gen_tgan = os.path.join(const.dir.data_gen(), "TGAN")
model_dir = os.path.join(const.dir.models(), "tgan")

batch_size = 500
retrain=True

In [3]:
import tensorflow as tf
print(tf.__version__)
tf.test.is_gpu_available()

1.15.5


True

In [12]:
?TGANModel

In [18]:
if retrain:
    tf.reset_default_graph()
    if os.path.exists(model_dir):
        shutil.rmtree(model_dir)
    timer_tgan = utils.timer.load(timer_path)
    timer_tgan.start("TGAN")
    tgan = TGANModel(columns_num, output = model_dir,
                     gpu="0",
                     max_epoch = 40, steps_per_epoch = data_train.shape[0] / batch_size, batch_size=batch_size,
                     num_dis_hidden = 256, num_dis_layers=2, z_dim=128,
                    restore_session = False, save_checkpoints = True)
    tgan.fit(data_train)
    timer_tgan.stop("TGAN")
    timer_tgan.save(timer_path)

PermissionError: [WinError 5] Ingen tilgang: 'S:/arneir/Master-thesis-storage/models\\tgan'

In [14]:
tgan.save(os.path.join(model_dir, "model.pkl"))

[0315 10:50:12 @model.py:813] Model saved successfully.


In [6]:
tgan = TGANModel.load(os.path.join(model_dir, "model.pkl"))












Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[0315 10:57:59 @registry.py:126] gen/LSTM/00/FC input: [500, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[0315 10:57:59 @registry.py:134] gen/LSTM/00/FC output: [500, 100]
[0315 10:57:59 @registry.py:126] gen/LSTM/00/FC2 input: [500, 100]
[0315 10:57:59 @registry.py:134] gen/LSTM/00/FC2 output: [500, 1]
[0315 10:57:59 @registry.py:126] gen/LSTM/01/FC input: [500, 100]
[0315 10:57:59 @registry.py:134] gen/LSTM/01/FC output: [500, 100]
[0315 10:57:59 @registry.py:126] gen/LSTM/01/FC2 input: [500, 100]
[0315 10:57:59 @registry.py:134] gen/LSTM/01/FC2 output: [500, 5]
[0315 10:57:59 @registry.py:126] gen/LSTM/01/FC3 input: [500, 5]

[0315 10:58:00 @registry.py:126] gen/LSTM/10/FC2 input: [500, 100]
[0315 10:58:00 @registry.py:134] gen/LSTM/10/FC2 output: [500, 5]
[0315 10:58:00 @registry.py:126] gen/LSTM/10/FC3 input: [500, 5]
[0315 10:58:00 @registry.py:134] gen/LSTM/10/FC3 output: [500, 100]
[0315 10:58:00 @registry.py:126] gen/LSTM/11/FC input: [500, 100]
[0315 10:58:00 @registry.py:134] gen/LSTM/11/FC output: [500, 100]
[0315 10:58:00 @registry.py:126] gen/LSTM/11/FC2 input: [500, 100]
[0315 10:58:00 @registry.py:134] gen/LSTM/11/FC2 output: [500, 2]
[0315 10:58:00 @registry.py:126] gen/LSTM/11/FC3 input: [500, 2]
[0315 10:58:00 @registry.py:134] gen/LSTM/11/FC3 output: [500, 100]
[0315 10:58:00 @registry.py:126] gen/LSTM/12/FC input: [500, 100]
[0315 10:58:00 @registry.py:134] gen/LSTM/12/FC output: [500, 100]
[0315 10:58:00 @registry.py:126] gen/LSTM/12/FC2 input: [500, 100]
[0315 10:58:00 @registry.py:134] gen/LSTM/12/FC2 output: [500, 1]
[0315 10:58:00 @registry.py:126] gen/LSTM/13/FC input: [500, 100]
[03

In [15]:
tgan_synthetic_data_train = tgan.sample(ceil(data_train.shape[0] / batch_size) * batch_size).head(data_train.shape[0])
tgan_synthetic_data_train.to_csv(os.path.join(dir_data_gen_tgan, "train_dataset.csv"), index=False)
tgan_synthetic_data_test = tgan.sample(ceil(data_test.shape[0] / batch_size) * batch_size).head(data_test.shape[0])
tgan_synthetic_data_test.to_csv(os.path.join(dir_data_gen_tgan, "train_dataset.csv"), index=False)

  6%|#####                                                                               |30/500[00:00<00:11,39.83it/s]


In [8]:
print(tgan_synthetic_data_train.shape[0], "vs", data_train.shape[0])
print(tgan_synthetic_data_test.shape[0], "vs", data_test.shape[0])

30162 vs 30162
15060 vs 15060


In [16]:
tgan_synthetic_data_train

,age,workclass,fnlwgt,education,educational.num,marital.status,occupation,race,gender,capital.gain,capital.loss,hours.per.week,native.country,income
0,56.420453,Self-emp-not-inc,14442.746892,HS-grad,9.062582,Married,Exec-managerial,White,Male,-0.002000,0.002000,53.888244,US,>50K
1,25.365530,Private,13045.426430,HS-grad,8.543215,Never-married,Adm-clerical,White,Male,-0.001999,0.001995,40.001716,US,<=50K
2,35.669026,Self-emp-not-inc,131569.054474,Some-college,9.569303,Divorced,Craft-repair,White,Male,-0.002000,0.001949,75.647998,US,>50K
3,72.819391,Private,60094.733618,Assoc-voc,3.226417,Never-married,Exec-managerial,White,Female,-0.002000,-0.001897,30.357612,Non-US,<=50K
4,36.968999,Private,22194.396616,HS-grad,8.642888,Divorced,Handlers-cleaners,White,Male,-0.002000,0.001975,40.000643,US,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,46.078009,Private,120776.562197,Bachelors,12.120272,Married,Machine-op-inspct,White,Male,274.274323,0.001921,39.999426,US,<=50K
30158,36.191944,Government,159833.161333,HS-grad,8.780404,Married,Machine-op-inspct,White,Male,-0.002000,0.001984,50.791902,US,>50K
30159,43.094701,Government,146723.984332,Some-college,9.684307,Divorced,Protective-serv,White,Female,-0.002000,0.001989,40.000888,US,<=50K
30160,58.642034,Private,141208.238251,Bachelors,12.964556,Married,Adm-clerical,White,Male,-0.002000,0.001974,80.767438,US,<=50K


In [17]:
import helpers
helpers.eval.fit_and_evaluate_xgboost(tgan_synthetic_data_train, data_test, tree_method="hist")

[11:12:16] WARNING: ..\src\learner.cc:541: 
Parameters: { enable_categorical } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




(0.7621513944223107, 0.7719992505709936)

In [4]:
from tqdm.auto import tqdm
n_synthetic_datasets = 10
n_synthetic_datasets_existing=0

dataset_dir_tgan = os.path.join(const.dir.data_gen(), "TGAN") 
model_temp_dir = os.path.join(const.dir.models(), "temp_model")
if os.path.exists(model_temp_dir):
    shutil.rmtree(model_temp_dir)
os.makedirs(dataset_dir_tgan, exist_ok = True)
for i in tqdm(range(n_synthetic_datasets_existing, n_synthetic_datasets), desc = "Generated datasets", disable = True):
    tf.reset_default_graph()
    model_temp_dir_i = os.path.join(model_temp_dir, f"TGAN{i}")
    tgan = TGANModel(columns_num, output = model_temp_dir_i,
                     gpu="0",
                     max_epoch = 100, steps_per_epoch = data_train.shape[0] / batch_size, batch_size=batch_size,
                     num_dis_hidden = 256, num_dis_layers=2, z_dim=128,
                    restore_session = False, save_checkpoints = True)
    tgan.fit(data_train.reset_index(drop=True))
    fake_train = tgan.sample(ceil(data_train.shape[0] / batch_size) * batch_size).head(data_train.shape[0])
    fake_train.to_csv(os.path.join(dataset_dir_tgan, f"gen{i}.csv"))
    del tgan




[0315 11:40:22 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...








Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[0315 11:40:22 @registry.py:126] gen/LSTM/00/FC input: [500, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[0315 11:40:22 @registry.py:134] gen/LSTM/00/FC output: [500, 100]
[0315 11:40:22 @registry.py:126] gen/LSTM/00/FC2 input: [500, 100]
[0315 11:40:22 @registry.py:134] gen/LSTM/00/FC2 output: [500, 1]
[0315 11:40:22 @registry.py:126] gen/LSTM/01/FC input: [500, 100]
[0315 11:40:22 @registry.py:134] gen/LSTM/01/FC output: [500, 100]
[0315 11:40:22 @registry.py:126] gen/LSTM/01/FC2 input: [500, 100]
[0315 11:40:22 @registry

[0315 11:40:23 @registry.py:134] gen/LSTM/10/FC2 output: [500, 5]
[0315 11:40:23 @registry.py:126] gen/LSTM/10/FC3 input: [500, 5]
[0315 11:40:23 @registry.py:134] gen/LSTM/10/FC3 output: [500, 100]
[0315 11:40:23 @registry.py:126] gen/LSTM/11/FC input: [500, 100]
[0315 11:40:23 @registry.py:134] gen/LSTM/11/FC output: [500, 100]
[0315 11:40:23 @registry.py:126] gen/LSTM/11/FC2 input: [500, 100]
[0315 11:40:23 @registry.py:134] gen/LSTM/11/FC2 output: [500, 2]
[0315 11:40:23 @registry.py:126] gen/LSTM/11/FC3 input: [500, 2]
[0315 11:40:23 @registry.py:134] gen/LSTM/11/FC3 output: [500, 100]
[0315 11:40:23 @registry.py:126] gen/LSTM/12/FC input: [500, 100]
[0315 11:40:23 @registry.py:134] gen/LSTM/12/FC output: [500, 100]
[0315 11:40:23 @registry.py:126] gen/LSTM/12/FC2 input: [500, 100]
[0315 11:40:23 @registry.py:134] gen/LSTM/12/FC2 output: [500, 1]
[0315 11:40:23 @registry.py:126] gen/LSTM/13/FC input: [500, 100]
[0315 11:40:23 @registry.py:134] gen/LSTM/13/FC output: [500, 100]
[03

[0315 11:40:29 @base.py:209] Setup callbacks graph ...






[0315 11:40:29 @summary.py:46] [MovingAverageSummary] 6 operations in collection 'MOVING_SUMMARY_OPS' will be run with session hooks.
[0315 11:40:29 @summary.py:93] Summarizing collection 'summaries' of size 9.

[0315 11:40:29 @graph.py:98] Applying collection UPDATE_OPS of 8 ops.

[0315 11:40:31 @base.py:230] Creating the session ...


Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.



[0315 11:40:34 @base.py:236] Initializing the session ...
[0315 11:40:34 @base.py:243] Graph Finalized.


[0315 11:40:34 @concurrency.py:38] Starting EnqueueThread QueueInput/input_queue ...

[0315 11:40:35 @base.py:275] Start Epoch 1 ...


 97%|##################################################################################1  |58/60[00:10<00:00, 7.27it/s]

100%|#####################################################################################|60/60[00:13<00:00, 4.39it/s]

...

[0315 15:17:31 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-5940.
[0315 15:17:33 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.888
[0315 15:17:33 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.9
[0315 15:17:33 @monitor.py:467] GAN_loss/discrim/loss: 0.3188
[0315 15:17:33 @monitor.py:467] GAN_loss/gen/final-g-loss: 6.094
[0315 15:17:33 @monitor.py:467] GAN_loss/gen/klloss: 0.7227
[0315 15:17:33 @monitor.py:467] GAN_loss/gen/loss: 5.3713
[0315 15:17:33 @monitor.py:467] QueueInput/queue_size: 50
[0315 15:17:33 @group.py:48] Callbacks took 4.066 sec in total. ModelSaver: 2.36 seconds; JSONWriter: 1.41 seconds
[0315 15:17:33 @base.py:275] Start Epoch 100 ...


100%|#####################################################################################|60/60[00:08<00:00, 7.06it/s]

[0315 15:17:41 @base.py:285] Epoch 100 (global_step 6000) finished, time:8.5 seconds.


[0315 15:17:44 @saver.py:79] Model saved to S:/arneir/Master-thesis-storage/models\temp_model\TGAN9\model\model-6000.
[0315 15:17:45 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.89
[0315 15:17:45 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.844
[0315 15:17:45 @monitor.py:467] GAN_loss/discrim/loss: 0.38187
[0315 15:17:45 @monitor.py:467] GAN_loss/gen/final-g-loss: 6.0567
[0315 15:17:45 @monitor.py:467] GAN_loss/gen/klloss: 0.19996
[0315 15:17:45 @monitor.py:467] GAN_loss/gen/loss: 5.8567
[0315 15:17:45 @monitor.py:467] QueueInput/queue_size: 50
[0315 15:17:45 @group.py:48] Callbacks took 3.853 sec in total. ModelSaver: 2.37 seconds; JSONWriter: 1.28 seconds
[0315 15:17:45 @base.py:289] Training has finished!
[0315 15:17:46 @input_source.py:178] EnqueueThread QueueInput/input_queue Exited.
[0315 15:17:47 @collection.py:146] New collections created in tower : tf.GraphKeys.REGULARIZATION_LOSSES
[0315 15:17:47 @collection.py:165] These collections were modified but restored in 

 12%|##########                                                                          |60/500[00:01<00:08,54.45it/s]
